In [1]:
import os
import sys
import numpy as np
import json
import matplotlib.pyplot as plt
#import mdtraj
import urllib.request
import yaml
import pprint
import pandas as pd
from pandas.io.json import json_normalize
import math
import numpy as np

import tabulate

sys.path.insert(1, '../../Databank/Scripts/BuildDatabank/')
from databankLibrary import download_link, lipids_dict, databank

path = '../../Databank/Data/Simulations/'
db_data = databank(path)
systems = db_data.get_systems()


/home/osollila/.conda/envs/py39/lib/python3.9/site-packages/MDAnalysis/coordinates/chemfiles.py:108: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  MIN_CHEMFILES_VERSION = LooseVersion("0.9")


In [10]:
CholPopcSystems = {}

for system in systems:
    
    # Select systems with only POPC and/or cholesterol
    skip = 0
    for molecule in system['COMPOSITION']:
        if molecule != 'SOL' and molecule != 'CHOL' and molecule != 'POPC':
            skip = 1
    if skip > 0:
        continue
    
    if system['TEMPERATURE'] < 298 or system['TEMPERATURE'] > 300:
        continue
    
    #print(system['COMPOSITION'])
    
    # Copy ASCII files for plotting. 
    # Requires that *dat files are in the core databank. 
    # These are produced by codes but not committed to git
    DBpath = system['path']
    NEWpath = DBpath.replace('../../Databank/Data/Simulations/','../DATA/ForPlotting/')
    !mkdir -p {NEWpath}
    !cp {DBpath}/*.dat {NEWpath}
    !cp {DBpath}/*.json {NEWpath}
    !cp {DBpath}/README.yaml {NEWpath}
    
    
    
    
    # Save information in a dictionary
    try:
        Nchol = np.sum(system['COMPOSITION']['CHOL']['COUNT'])
    except:
        Nchol = 0
    Npopc = np.sum(system['COMPOSITION']['POPC']['COUNT'])
    CholFraction = Nchol/(Nchol+Npopc)
    ForceField = system['FF']
    temperature = system['TEMPERATURE']
    
    
    if ForceField not in CholPopcSystems.keys():
        CholPopcSystems[ForceField] = []
    save_dict = {'CholFraction': CholFraction,
                'ForceField': ForceField,
                'temperature': temperature,
                'path': NEWpath,
                'DOI:': system['DOI']
                }
    CholPopcSystems[ForceField].append(save_dict)

In [26]:
CholPopcSystems.keys()

dict_keys(['MacRog', 'Lipid17', 'Slipids', 'ECC-lipids', 'Orange', 'Berger delivered by Peter Tieleman', 'CHARMM36', 'OPLS3e', 'Berger and Modified Höltje model for cholesterol', 'Chiu Gromos', 'Berger', 'slipids', 'GROMOS-CKP', 'Lipid14 with Verlet scheme'])

In [16]:
for FF in CholPopcSystems:
    print(FF)
    df = pd.DataFrame.from_records(CholPopcSystems[FF])
    print(df)
    df.to_csv('../DATA/ForPlotting/'+FF+'.txt', sep = ' ')

MacRog
    CholFraction ForceField  temperature  \
0       0.288889     MacRog       298.15   
1       0.000000     MacRog       298.15   
2       0.111111     MacRog       298.15   
3       0.000000     MacRog       298.00   
4       0.000000     MacRog       300.00   
5       0.111111     MacRog       298.15   
6       0.000000     MacRog       298.00   
7       0.000000     MacRog       298.00   
8       0.466667     MacRog       298.15   
9       0.000000     MacRog       298.15   
10      0.000000     MacRog       298.00   
11      0.000000     MacRog       298.15   
12      0.200000     MacRog       298.15   
13      0.384615     MacRog       298.15   
14      0.000000     MacRog       298.00   
15      0.288889     MacRog       298.15   
16      0.111111     MacRog       298.15   
17      0.384615     MacRog       298.15   
18      0.466667     MacRog       298.15   
19      0.200000     MacRog       298.15   
20      0.466667     MacRog       298.15   
21      0.288889     MacR

In [36]:
!pwd

/media/osollila/Data/NMRlipids/NmrLipidsCholXray/scripts


In [37]:
!ls ../../Databank/Data/Simulations/cfb/a18/cfba18cb6fbbc3ce348e83cc27dead0472882f17/53a843832460b6e8320a71cad1a51ef0a49b3f06/

 25.tpr		     LipidDensity_no_center.dat
 25WperL_50ns.xtc    POPCOrderParameters.dat
 apl.json	     POPCOrderParameters.json
 centered.xtc	     POPC_OrderParameters_quality.json
 conf.gro	     README.yaml
'#conf.gro.1#'	     thickness.json
 conf.pdb	     TotalDensity.dat
 FormFactor.dat      TotalDensity.json
 FormFactor.json     WaterDensity.dat
 LipidDensity.dat    WaterDensity.json
 LipidDensity.json
